In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import KNNImputer 
from sklearn.preprocessing import OneHotEncoder, scale, LabelEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import pickle

In [4]:
with open(r'/Users/mak/Desktop/Code_With_Me/Default/Default/model.pkl', 'rb') as f:
    model = pickle.load(f)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.5.1 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
with open(r'/Users/mak/Desktop/Code_With_Me/Default/Default/ohe.pkl', 'rb') as f:
    encoder = pickle.load(f)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.5.1 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
with open(r'/Users/mak/Desktop/Code_With_Me/Default/Default/scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.5.1 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
data = pd.read_csv(r'/Users/mak/Desktop/Code_With_Me/Default/Default/df3.csv')

In [8]:
cat_vars = []
num_vars = []
for i in data.columns:
    if(data[i].dtype == "object"):
        cat_vars.append(i)
    else:
        num_vars.append(i)
print("Categorical Variables:\n",cat_vars,"\n")
print("Numerical Variables:\n",num_vars,"\n")

Categorical Variables:
 ['loan_limit', 'Gender', 'approv_in_adv', 'loan_type', 'loan_purpose', 'Credit_Worthiness', 'open_credit', 'business_or_commercial', 'Neg_ammortization', 'interest_only', 'lump_sum_payment', 'occupancy_type', 'total_units', 'credit_type', 'co-applicant_credit_type', 'age', 'submission_of_application', 'Region'] 

Numerical Variables:
 ['loan_amount', 'rate_of_interest', 'Interest_rate_spread', 'Upfront_charges', 'term', 'property_value', 'income', 'Credit_Score', 'LTV', 'dtir1'] 



In [9]:
cat_data = data[cat_vars]

In [10]:
num_data = data[num_vars]

In [11]:
encoded_cat = encoder.transform(cat_data)

In [12]:
transformed_ohe = pd.DataFrame(
    data=encoded_cat,
    columns=encoder.get_feature_names_out(cat_vars),
    index=data.index,
)
transformed_ohe.head()

,loan_limit_cf,loan_limit_ncf,Gender_Female,Gender_Joint,Gender_Male,Gender_Sex Not Available,approv_in_adv_nopre,approv_in_adv_pre,loan_type_type1,loan_type_type2,...,age_55-64,age_65-74,age_<25,age_>74,submission_of_application_not_inst,submission_of_application_to_inst,Region_North,Region_North-East,Region_central,Region_south
0,1,0,0,0,1,0,1,0,1,0,...,0,0,0,0,1,0,1,0,0,0
1,1,0,0,0,1,0,1,0,1,0,...,0,0,0,0,1,0,0,0,0,1
2,1,0,0,1,0,0,1,0,1,0,...,0,0,0,0,0,1,1,0,0,0
3,1,0,0,0,0,1,0,1,1,0,...,0,0,0,0,0,1,0,0,0,1
4,1,0,0,1,0,0,1,0,1,0,...,0,0,0,0,1,0,1,0,0,0


In [13]:
test_data = pd.concat([num_data, transformed_ohe], axis=1)

In [14]:
test_data

,loan_amount,rate_of_interest,Interest_rate_spread,Upfront_charges,term,property_value,income,Credit_Score,LTV,dtir1,...,age_55-64,age_65-74,age_<25,age_>74,submission_of_application_not_inst,submission_of_application_to_inst,Region_North,Region_North-East,Region_central,Region_south
0,336500.0,3.625000,0.338900,5366.480000,360.0,358000.0,5760.0,775.0,93.994413,41.000000,...,0,0,0,0,1,0,1,0,0,0
1,66500.0,4.750000,1.378400,1741.580000,180.0,188000.0,5760.0,610.0,35.372340,36.000000,...,0,0,0,0,1,0,0,0,0,1
2,196500.0,3.625000,0.736100,4778.260000,240.0,268000.0,8700.0,826.0,73.320896,22.000000,...,0,0,0,0,0,1,1,0,0,0
3,446500.0,3.708333,0.033633,4446.666667,180.0,968000.0,15300.0,759.0,46.126033,20.000000,...,0,0,0,0,0,1,0,0,0,1
4,256500.0,4.625000,0.357000,2791.170000,360.0,288000.0,5880.0,652.0,89.062500,40.000000,...,0,0,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118664,246500.0,3.750000,0.415000,1797.886667,360.0,298000.0,3960.0,730.0,83.729917,41.333333,...,1,0,0,0,0,1,0,0,0,1
118665,706500.0,3.990000,0.343000,1050.000000,360.0,948000.0,12600.0,540.0,74.525316,31.000000,...,0,0,0,0,1,0,0,0,0,1
118666,386500.0,3.750000,0.059800,3098.750000,180.0,1008000.0,8640.0,890.0,38.343254,45.000000,...,0,1,0,0,0,1,0,0,0,1
118667,726500.0,4.560000,0.491300,1567.370000,360.0,1128000.0,15300.0,868.0,64.406028,43.000000,...,0,0,0,0,1,0,1,0,0,0


In [15]:
scaled_data = scaler.transform(test_data)

In [16]:
predictions = model.predict(scaled_data)

In [21]:
predic = pd.DataFrame(predictions, columns=["prediction"])
predic

,prediction
0,0
1,0
2,0
3,0
4,0
...,...
118664,1
118665,0
118666,0
118667,0


In [22]:
predic.value_counts()

prediction
0             99208
1             19461
Name: count, dtype: int64